In [1]:
import syft as sy
from syft.core.node.new.api import NodeView

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/dataset.py -- dk: creating a dataset
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/dataset.py -- dk: creating dataset
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/dataset.py -- dk: converting createasset to asset
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/dataset.py -- dk: transforming createdataset to dataset
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/project.py --- dk: get data subject registry (Project)
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/project.py --- dk: creating a project (ProjectSubmit)


In [2]:
worker = sy.Worker.named("test-domain-1", processes=1)
domain_client = worker.root_client

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/worker.py --- dk: naming a worker
SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.task.oblv_service.OblvService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <c

In [3]:
results = domain_client.api.services.dataset.get_all()
results

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


,type,id,name,url
0,syft.core.node.new.dataset.Dataset,693d75b4f23842bbbb2686a64cc5b339,Canada Trade Value,https://github.com/OpenMined/datasets/tree/mai...


In [4]:
assert len(results) == 1

In [5]:
dataset = results[-1]

In [6]:
dataset

```python
Syft Dataset: Canada Trade Value
Assets:
	canada_trade_flow: all the datas
Citation: Person, place or thing
URL: https://github.com/OpenMined/datasets/tree/main/trade_flow
Description: Canada Trade Data

```

In [7]:
dataset.assets

,key,type,id
0,canada_trade_flow,syft.core.node.new.dataset.Asset,cb20b2c29fe44876bd985d0595c587f6


In [8]:
mock = dataset.assets[0].mock
mock

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
10,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Bangladesh,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,227222,0
11,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Haiti,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,14748,0
12,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Guatemala,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1314,0
13,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Iraq,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1825,0
14,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Israel,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1063627,0
15,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Italy,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,8359327,0
16,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Jordan,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,16858,0
17,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Rep. of Moldova,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,29897,0
18,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Nigeria,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,22235,0
19,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Slovenia,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1328,0


In [9]:
mock["Trade Value (US$)"].sum()

9738381

In [10]:
@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


In [11]:
result = sum_trade_value_mil(trade_data=mock)
result

(9.738381, -13.437739492803672)

In [12]:
assert result[0] == 9.738381

In [13]:
assert isinstance(result[1], float)

In [14]:
assert len(sum_trade_value_mil.kwargs) == 1

In [15]:
domain_client.api.node_name

'test-domain-1'

In [16]:
domain_client.api.signing_key.verify_key

aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f

In [17]:
node_view = NodeView(node_name=domain_client.api.node_name, verify_key=domain_client.api.signing_key.verify_key)
assert node_view in sum_trade_value_mil.kwargs

`NodeView` is a schema based on `pydantic`'s `BaseModel`

```python
@serializable(recursive_serde=True)
class NodeView(BaseModel):
    class Config:
        arbitrary_types_allowed = True

    node_name: str
    verify_key: SyftVerifyKey

    @staticmethod
    def from_api(api: SyftAPI):
        # stores the name root verify key of the domain node
        node_metadata = api.connection.get_node_metadata(api.signing_key)
        return NodeView(
            node_name=node_metadata.name,
            verify_key=SyftVerifyKey.from_string(node_metadata.verify_key),
        )

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, NodeView):
            return False
        return self.node_name == other.node_name and self.verify_key == other.verify_key

    def __hash__(self) -> int:
        return hash((self.node_name, self.verify_key))
```

In [18]:
assert "trade_data" in sum_trade_value_mil.kwargs[node_view]

In [19]:
sum_trade_value_mil.kwargs

{NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'trade_data': <UID: d8d0076f13804c6b87ae57966f610722>}}

In [20]:
assert len(sum_trade_value_mil.outputs) == 0

In [21]:
assert sum_trade_value_mil.input_policy.inputs[node_view]["trade_data"] == mock.id

In [22]:
sum_trade_value_mil.code

'@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef sum_trade_value_mil(trade_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    df = trade_data\n    total = df["Trade Value (US$)"].sum()\n    return (float(total / 1_000_000), float(noise))\n'

In [23]:
guest_domain_client = domain_client.guest()

In [34]:
guest_credentials = guest_domain_client.credentials
guest_credentials

<SigningKey for: bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935>

In [25]:
# guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")

<class 'syft.core.node.new.response.SyftError'>: User already exists with email: jane@caltech.edu

In [35]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: PythonConnection>

In [36]:
assert guest_domain_client.credentials != guest_credentials

AssertionError: 

In [33]:
guest_domain_client.api.services.code.request_code_execution(sum_trade_value_mil)

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


<class 'syft.core.node.new.response.SyftError'>: Duplication Key Error: id=<UID: 5e61a783fdb34dfbb3a6858f3fd9ce12> user_verify_key=bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935 raw_code='@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef sum_trade_value_mil(trade_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    df = trade_data\n    total = df["Trade Value (US$)"].sum()\n    return (float(total / 1_000_000), float(noise))\n' input_policy=syft.core.node.new.user_code.ExactMatch output_policy=syft.core.node.new.user_code.SingleExecutionExactOutput output_policy_state=syft.core.node.new.user_code.OutputPolicyStateExecuteOnce parsed_code="def user_func_sum_trade_value_mil_bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0(trade_data):\n\n    def sum_trade_value_mil(trade_data):\n        import pandas as pd\n        from opendp.mod import enable_features\n        enable_features('contrib')\n        from opendp.measurements import make_base_laplace\n        aggregate = 0.0\n        base_lap = make_base_laplace(scale=5.0)\n        noise = base_lap(aggregate)\n        df = trade_data\n        total = df['Trade Value (US$)'].sum()\n        return (float(total / 1000000), float(noise))\n    result = sum_trade_value_mil(trade_data=trade_data)\n    return result" service_func_name='sum_trade_value_mil' unique_func_name='user_func_sum_trade_value_mil_bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0' user_unique_func_name='user_func_sum_trade_value_mil_bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935' code_hash='5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0' signature=<Signature (trade_data) -> Dict[str, Any]> status={NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>} enclave_metadata=None

In [29]:
guest_domain_client.api.services.code.request_code_execution??

Signature:
guest_domain_client.api.services.code.request_code_execution(
    code: syft.core.node.new.user_code.SubmitUserCode,
) -> Union[syft.core.node.new.response.SyftSuccess, syft.core.node.new.response.SyftError]
Docstring: Request Code execution on user code
Source:   
    def wrapper(*args, **kwargs):
        print(f"{__file__} --- dk: api wrapper function for generating remote function")
        blocking = True
        if "blocking" in kwargs:
            blocking = bool(kwargs["blocking"])
            del kwargs["blocking"]

        _valid_kwargs = {}
        if "kwargs" in signature.parameters:
            _valid_kwargs = kwargs
        else:
            for key, value in kwargs.items():
                if key not in signature.parameters:
                    return SyftError(
                        message=f"""Invalid parameter: `{key}`. Valid Parameters: {list(signature.parameters)}"""
                    )
                param = signature.parameters[key]
                

Note: The `wrapper` function are related to the functions in `PySyft/packages/syft/src/syft/core/node/new/user_code_service.py`

In [41]:
guest_domain_client.code

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


,type,id,status,service_func_name
0,syft.core.node.new.user_code.UserCode,17a383fea3264e9fabee57302b7f06a7,"{NodeView(node_name='test-domain-1', verify_ke...",sum_trade_value_mil


In [42]:
assert len(guest_domain_client.api.services.code.get_all()) == 1

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


In [43]:
new_project = sy.Project(name="My Cool UN Project")

In [44]:
new_project??

Type:           ProjectSubmit
String form:    id=None name='My Cool UN Project' description=None changes=[]
File:           ~/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/project.py
Source:        
@serializable(recursive_serde=True)
class ProjectSubmit(SyftObject):
    __canonical_name__ = "ProjectSubmit"
    __version__ = SYFT_OBJECT_VERSION_1
    print(f"{__file__} --- dk: creating a project (ProjectSubmit)")

    id: Optional[UID]
    name: str
    description: Optional[str]
    changes: List[ObjectPermissionChange] = []

    __attr_repr_cols__ = ["name", "changes"]

    def set_description(self, description: str, msg: Optional[str] = None) -> None:
        self.description = description

    def add_request(self, obj: SyftObject, permission: Enum) -> None:
        change = ObjectPermissionChange(
            object_uid=obj.id, object_type=type(obj), permission=permission
        )
        self.changes.append(change)
Init docstring:
Create a new model by parsing and

In [45]:
proj_desc = """Hi, I want to calculate the trade volume in million's with my cool code."""
new_project.set_description(proj_desc)

In [46]:
submitted_code = guest_domain_client.code[0]

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


In [48]:
submitted_code??

Type:           UserCode
String form:    id=<UID: 17a383fea3264e9fabee57302b7f06a7> user_verify_key=bddb6dccad9d957b148f0d9bbf7e95da26088f5fa5126afbfc221fa01cc47935 raw_code='@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef sum_trade_value_mil(trade_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    df = trade_data\n    total = df["Trade Value (US$)"].sum()\n    return (float(total / 1_000_000), float(noise))\n' input_policy=syft.core.node.new.user_code.ExactMatch output_policy=syft.core.node.new.user_code.SingleExecutionExactOutput output_policy_state=syft.core.node.new.user_code.OutputPolicyStateExecuteOnce parsed_code="def user_func_sum_trade_value_mil_bddb6dccad9d957b148f0d9bbf7e95da2

In [49]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [50]:
new_project.add_request??

Signature:
new_project.add_request(
    obj: syft.core.node.common.node_table.syft_object.SyftObject,
    permission: enum.Enum,
) -> None
Docstring: <no docstring>
Source:   
    def add_request(self, obj: SyftObject, permission: Enum) -> None:
        change = ObjectPermissionChange(
            object_uid=obj.id, object_type=type(obj), permission=permission
        )
        self.changes.append(change)
File:      ~/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/project.py
Type:      method

In [51]:
new_project.changes

,type,id
0,syft.core.node.new.project.ObjectPermissionChange,25c17e8aaf9a472e8324be862ad3bcc7


In [52]:
new_project

```python
class ProjectSubmit:
  id: str = None
  name: str = "My Cool UN Project"
  description: str = "Hi, I want to calculate the trade volume in million's with my cool code."
  changes: str = [syft.core.node.new.project.ObjectPermissionChange]

```

In [53]:
guest_domain_client.submit_project(new_project)

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function
/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/response.py --- dk: success!


<class 'syft.core.node.new.response.SyftSuccess'>: Project Submitted

In [54]:
guest_domain_client.submit_project??

Type:        property
String form: <property object at 0x7f60b533e250>
Source:     
# guest_domain_client.submit_project.fget
@property
def submit_project(self) -> Callable:
    if self.api is not None and hasattr(self.api.services, "project"):
        return self.api.services.project.submit
    return None

In [55]:
guest_domain_client._api = None
_ = guest_domain_client.api

In [56]:
result = guest_domain_client.api.services.code.sum_trade_value_mil(trade_data=mock.id)
result

/home/dk/Desktop/projects/PySyft/packages/syft/src/syft/core/node/new/api.py --- dk: api wrapper function for generating remote function


<class 'syft.core.node.new.response.SyftNotReady'>: <class 'syft.core.node.new.user_code.UserCode'> Your code is waiting for approval: {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}

In [57]:
guest_domain_client.api.services.code.sum_trade_value_mil??

Signature: guest_domain_client.api.services.code.sum_trade_value_mil(trade_data) -> Dict[str, Any]
Docstring: Users custom func sum_trade_value_mil
Source:   
    def wrapper(*args, **kwargs):
        print(f"{__file__} --- dk: api wrapper function for generating remote function")
        blocking = True
        if "blocking" in kwargs:
            blocking = bool(kwargs["blocking"])
            del kwargs["blocking"]

        _valid_kwargs = {}
        if "kwargs" in signature.parameters:
            _valid_kwargs = kwargs
        else:
            for key, value in kwargs.items():
                if key not in signature.parameters:
                    return SyftError(
                        message=f"""Invalid parameter: `{key}`. Valid Parameters: {list(signature.parameters)}"""
                    )
                param = signature.parameters[key]
                if isinstance(param.annotation, str):
                    # 🟡 TODO 21: make this work for weird string type situations

In [59]:
assert isinstance(result, sy.SyftNotReady)  # waiting to be approved

### Notebook 1 Summary:



First we get a `Worker` and `domain_client` with
```python
worker = sy.Worker.named("test-domain-1", processes=1)
domain_client = worker.root_client
```

We can access the datasets on the domain with `results = domain_client.api.services.dataset.get_all()` and its mock dataset with `mock = dataset.assets[0].mock`. 

Now, let's try to do a sum on `Trade Value` with `mock["Trade Value (US$)"].sum()`. Asssume we have the data locally, then the result sum is `9738381`. 

However, let's do private data science with differential privacy.
```python
@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))
```

Running this function with `result = sum_trade_value_mil(trade_data=mock)` will create the result of `(sum, noise) = (9.738381, -13.437739492803672)`

How about remote data science with `syft`?

Creating a guest with 
```python
guest_domain_client = domain_client.guest()
guest_credentials = guest_domain_client.credentials
```
If not registered, then do it:
```python
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")
```
Then log in:
```python
guest_domain_client.login(email="jane@caltech.edu", password="abc123")
```

Then the guest request to execute the `sum_trade_value_mil` function
```python
guest_domain_client.api.services.code.request_code_execution(sum_trade_value_mil)
```
The guest can check the code with `guest_domain_client.code`

Now we create a project and 

```python
new_project = sy.Project(name="My Cool UN Project")
proj_desc = """Hi, I want to calculate the trade volume in million's with my cool code."""
new_project.set_description(proj_desc)